In [1]:
# region General Imports
import os
import re
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

import tempfile
import tensorflow as tf
import numpy as np
from tensorflow import keras
%load_ext tensorboard
    
# os.environ["KERAS_BACKEND"] = "tf"
# os.environ["TF_USE_LEGACY_KERAS"] = "0"
# import jax
# import jax.numpy as jnp
# import keras
    
# region Keras
from keras.models import Model
from keras.layers import Dense, Activation, Permute, Dropout
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    Conv1D,
    MaxPooling1D,
    AveragePooling1D,
)
from keras.layers import SeparableConv2D, DepthwiseConv2D
from keras.layers import BatchNormalization
from keras.layers import SpatialDropout2D
from keras.regularizers import l1_l2
from keras.layers import Input, Flatten
from keras.constraints import max_norm
from keras import backend as K
# endregion Keras

from custom_datasets.fatigue_mi import FatigueMI

# Sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, BatchNormalization
from sklearn import preprocessing

from model_optim.utils import channels_to_channels_idx

/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-24 15:26:06.323201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 15:26:06.323270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 15:26:06.324638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 15:26:06.331994: I tensorflow/core/platform/cpu_feature_guard

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [2]:
SKLRNG = 42
# RNG = jax.random.PRNGKey(SKLRNG)

In [3]:
subject_best_trials = glob.glob('./temp/**/model/study_best_trial.npy', recursive=True)
subject_best_trials = sorted(subject_best_trials, key=lambda x: os.path.getmtime(x))

subject_all_trials = glob.glob('./temp/**/model/study.npy', recursive=True)
subject_all_trials = sorted(subject_all_trials, key=lambda x: os.path.getmtime(x))

rpprint(subject_best_trials)

['./temp/[1]/8ca44c9b9c7c4410b37ac5781bd7da1f/model/study_best_trial.npy']

In [4]:
# region Helper funcs
def shallow_conv_net_square_layer(x):
    return tf.math.square(x)

def shallow_conv_net_log_layer(x):
    return tf.math.log(tf.clip_by_value(x, 1e-7, 10000))

CUSTOM_OBJECTS = {
    "shallow_conv_net_square_layer": shallow_conv_net_square_layer, 
    "shallow_conv_net_log_layer": shallow_conv_net_log_layer 
}
# endregion Helper funcs

# region Models
def shallow_conv_net(
    nb_classes, channels, samples, **kwargs
):
    """
    From: https://github.com/vlawhern/arl-eegmodels/blob/master/EEGModels.py
    """

    _POOL_SIZE_D2_ = kwargs.get("pool_size_d2", 35)
    _STRIDES_D2_ = kwargs.get("strides_d2", 7)
    _CONV_FILTERS_D2_ = kwargs.get("conv_filters_d2", 13)

    _POOL_SIZE_ = kwargs.get("pool_size", (1, _POOL_SIZE_D2_))
    _STRIDES_ = kwargs.get("strides", (1, _STRIDES_D2_))
    _CONV_FILTERS_ = kwargs.get("conv_filters", (1, _CONV_FILTERS_D2_))

    _CONV2D_1_UNITS_ = kwargs.get("conv2d_1_units", 40)
    _CONV2D_2_UNITS_ = kwargs.get("conv2d_2_units", 40)
    _L2_REG_1_ = kwargs.get("l2_reg_1", 0.01)
    _L2_REG_2_ = kwargs.get("l2_reg_2", 0.01)
    _L2_REG_3_ = kwargs.get("l2_reg_3", 0.01)
    _DROPOUT_RATE_ = kwargs.get("dropout_rate", 0.5)

    input_main = Input(shape=(channels, samples, 1))
    block1 = Conv2D(
        _CONV2D_1_UNITS_,
        _CONV_FILTERS_,
        input_shape=(channels, samples, 1),
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_1_),
    )(input_main)
    # block1       = Conv2D(40, (channels, 1), use_bias=False,
    #                       kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1 = Conv2D(
        _CONV2D_2_UNITS_,
        (channels, 1),
        use_bias=False,
        kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_2_),
    )(block1)
    block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1 = Activation(shallow_conv_net_square_layer)(block1)
    block1 = AveragePooling2D(pool_size=_POOL_SIZE_, strides=_STRIDES_)(block1)
    block1 = Activation(shallow_conv_net_log_layer)(block1)
    block1 = Dropout(_DROPOUT_RATE_)(block1)
    flatten = Flatten()(block1)
    # dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    dense = Dense(
        nb_classes,
        kernel_constraint=max_norm(0.5),
        kernel_regularizer=keras.regularizers.L2(_L2_REG_3_),
    )(flatten)
    softmax = Activation("softmax")(dense)

    return Model(inputs=input_main, outputs=softmax)

# endregion Models

In [5]:
def data_generator(dataset, subjects = [1], channel_idx = [], filters = ([8, 32],), sfreq = 250):

    find_events = lambda raw, event_id: mne.find_events(raw, shortest_event=0, verbose=False) if len(mne.utils._get_stim_channel(None, raw.info, raise_error=False)) > 0 else mne.events_from_annotations(raw, event_id=event_id, verbose=False)[0]
    
    data = dataset.get_data(subjects=subjects)
    
    X = []
    y = []
    metadata = []

    for subject_id in data.keys():
        for session_id in data[subject_id].keys():
            for run_id in data[subject_id][session_id].keys():
                raw = data[subject_id][session_id][run_id]
                
                for fmin, fmax in filters:
                    raw = raw.filter(l_freq = fmin, h_freq = fmax, method = 'iir', picks = 'eeg', verbose = False)
                
                events = find_events(raw, dataset.event_id)

                tmin = dataset.interval[0]
                tmax = dataset.interval[1]

                channels = np.asarray(raw.info['ch_names'])[channel_idx] if len(channel_idx) > 0 else np.asarray(raw.info['ch_names'])

                # rpprint(channels)
                
                stim_channels = mne.utils._get_stim_channel(None, raw.info, raise_error=False)
                picks = mne.pick_channels(raw.info["ch_names"], include=channels, exclude=stim_channels, ordered=True)

                x = mne.Epochs(
                    raw,
                    events,
                    event_id=dataset.event_id,
                    tmin=tmin,
                    tmax=tmax,
                    proj=False,
                    baseline=None,
                    preload=True,
                    verbose=False,
                    picks=picks,
                    event_repeated="drop",
                    on_missing="ignore",
                )
                x_events = x.events
                inv_events = {k: v for v, k in dataset.event_id.items()}
                labels = [inv_events[e] for e in x_events[:, -1]]

                # rpprint({
                #     "X": np.asarray(x.get_data(copy=False)).shape,
                #     "y": np.asarray(labels).shape,
                #     "channels selected": np.asarray(raw.info['ch_names'])[channel_idx]
                # })

                # x.plot(scalings="auto")
                # display(x.info)
                
                x_resampled = x.resample(sfreq) # Resampler_Epoch
                x_resampled_data = x_resampled.get_data(copy=False) # Convert_Epoch_Array
                x_resampled_data_standard_scaler = np.asarray([
                    StandardScaler().fit_transform(x_resampled_data[i])
                    for i in np.arange(x_resampled_data.shape[0])
                ]) # Standard_Scaler_Epoch

                # x_resampled.plot(scalings="auto")
                # display(x_resampled.info)

                n = x_resampled_data_standard_scaler.shape[0]
                # n = x.get_data(copy=False).shape[0]
                met = pd.DataFrame(index=range(n))
                met["subject"] = subject_id
                met["session"] = session_id
                met["run"] = run_id
                x.metadata = met.copy()
                
                # X.append(x_resampled_data_standard_scaler)
                X.append(x)
                y.append(labels)
                metadata.append(met)

    return np.concatenate(X, axis=0), np.concatenate(y), pd.concat(metadata, ignore_index=True)

fat_dataset = FatigueMI()

In [6]:
def get_results_df(study):
        trial_metrics_dict = {
            "subjects": [],
            "model": [],
            "train_acc": [],
            "test_acc": [],
            "val_acc": [],
            "train_val_acc_diff": [],
            "train_loss": [],
            "val_loss": [],
            "train_val_loss_diff": [],
            "test_loss": [],
            "scores": [],
            "channels_selected": [],
            "num_channels_selected": [],
            "sfreq": [],
            "batch_size": [],
            "num_training_epochs": [],
            "train_time": [],
            "inf_time": [],
        }
        # for res_file_name in res_files:
        #     res = np.load(res_file_name, allow_pickle=True).item()
        #     study = res["study"]
        for i, trial in enumerate(study.trials_dataframe().itertuples()):
            trial_user_attrs = trial.user_attrs_trial_data
            trial_metrics_dict["scores"].append(trial.value)
            trial_metrics_dict["subjects"].append(trial_user_attrs["subjects"])
            trial_metrics_dict["model"].append(trial_user_attrs["model_name"]) if "model_name" in trial_user_attrs else None
            trial_metrics_dict["train_time"].append(trial_user_attrs["training_time"]) if "training_time" in trial_user_attrs else None
            trial_metrics_dict["inf_time"].append(trial_user_attrs["inference_time"]) if "inference_time" in trial_user_attrs else None
            trial_metrics_dict["num_training_epochs"].append(trial_user_attrs["num_training_epochs"] if "num_training_epochs" in trial_user_attrs else None)
            trial_metrics_dict["train_acc"].append(
                np.max(trial_user_attrs["train_accuracy"])
            )
            trial_metrics_dict["test_acc"].append(trial_user_attrs["test_accuracy"])
            trial_metrics_dict["val_acc"].append(
                np.max(trial_user_attrs["val_accuracy"])
            )
            trial_metrics_dict["channels_selected"].append(
                trial_user_attrs["channels_selected"]
            )
            trial_metrics_dict["num_channels_selected"].append(
                len(trial_user_attrs["channels_selected"])
            )
            trial_metrics_dict["train_val_acc_diff"].append(
                abs(
                    np.max(trial_user_attrs["train_accuracy"])
                    - np.max(trial_user_attrs["val_accuracy"])
                )
            )
            (
                trial_metrics_dict["train_loss"].append(
                    np.min(trial_user_attrs["train_loss"])
                    if "train_loss" in trial_user_attrs
                    else None
                )
            )
            (
                trial_metrics_dict["val_loss"].append(
                    np.min(trial_user_attrs["val_loss"])
                    if "val_loss" in trial_user_attrs
                    else None
                )
            )
            (
                trial_metrics_dict["train_val_loss_diff"].append(
                    abs(
                        np.min(trial_user_attrs["train_loss"])
                        - np.min(trial_user_attrs["val_loss"])
                    )
                    if "train_loss" in trial_user_attrs
                    and "val_loss" in trial_user_attrs
                    else None
                )
            )
            (
                trial_metrics_dict["test_loss"].append(
                    np.min(trial_user_attrs["test_loss"])
                    if "test_loss" in trial_user_attrs
                    else None
                )
            )
            trial_metrics_dict["sfreq"].append(
                trial.params_sfreq if hasattr(trial, "params_sfreq") else (trial_user_attrs["sfreq"] if "sfreq" in trial_user_attrs else None)
            )
            trial_metrics_dict["batch_size"].append(
                trial.params_batch_size if hasattr(trial, "params_batch_size") else (trial_user_attrs["batch_size"] if "batch_size" in trial_user_attrs else None)
            )
        # Delete any entries in the dictionary that are empty lists
        trial_metrics_dict = {
            k: v
            for k, v in trial_metrics_dict.items()
            if len([x for x in v if x is not None]) > 0
        }
        
        trial_metrics_df = pd.DataFrame(trial_metrics_dict)
        return trial_metrics_df

In [7]:
get_results_df(
    np.load(subject_all_trials[0], allow_pickle=True).item()
).sort_values(by="scores", ascending=True).head(5)

,subjects,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,num_channels_selected,sfreq,batch_size
9,[1],0.867647,0.681818,0.722222,0.145425,10.666684,10.572948,0.093737,10.993606,0.077611,"[P3, F4, P4, Pz, O1, F7, F8, A2, T6]",9,300,160
7,[1],0.838235,0.545455,0.666667,0.171569,13.874846,12.969428,0.905418,13.003958,0.111511,"[F3, F4, C4, P4, T3, O1, F8, T4]",8,128,32
15,[1],1.000000,0.500000,0.611111,0.388889,10.528308,10.074472,0.453835,12.001544,0.151585,"[P3, C4, P4, O2, F7, F8, A2]",7,128,96
14,[1],0.823529,0.500000,0.611111,0.212418,66.014801,62.986416,3.028385,63.090633,0.151735,"[P3, C3, F3, F4, C4, P4, Fp1, O2, T6, T4]",10,300,128
8,[1],0.852941,0.590909,0.555556,0.297386,24.199411,22.264116,1.935295,22.316408,0.197981,"[F3, Cz, Pz, T3, O1, O2, F8, A2, T4]",9,128,256


In [8]:
model = np.load(subject_best_trials[0], allow_pickle=True).item()
model_info = {
    "subject": model.user_attrs["trial_data"]["subject"] if hasattr(model.user_attrs["trial_data"], "subject") else int(re.compile(r"\[.*\]").search(model.user_attrs["trial_data"]["data_path"]).group(0).strip("[]")),
    "sfreq": model.params["sfreq"] if "sfreq" in model.params else 128,
    "batch_size": model.params["batch_size"] if "batch_size" in model.params else 128,
    "channels_selected": model.user_attrs["trial_data"]["channels_selected"],
    "channels_idx_selected": channels_to_channels_idx(model.user_attrs["trial_data"]["channels_selected"], fat_dataset.get_data(subjects=[1])[1]['0']['0'].info['ch_names'][:-1]),
    "model": tf.keras.models.model_from_json(model.user_attrs["trial_data"]["model"], custom_objects=CUSTOM_OBJECTS),
    "test_acc": model.user_attrs["trial_data"]["test_accuracy"]
}
if "weights" in model.user_attrs["trial_data"]:
    model_info["model"].set_weights(model.user_attrs["trial_data"]["weights"])
elif "model_weights" in model.user_attrs["trial_data"]:
    model_info["model"].set_weights(model.user_attrs["trial_data"]["model_weights"])

rpprint(model_info)

2024-03-24 15:26:12.093527: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 15:26:12.128354: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 15:26:12.128421: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 15:26:12.130920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 15:26:12.131091: I external/local_xla/xla/stream_executor

{
│   'subject': 1,
│   'sfreq': 300,
│   'batch_size': 160,
│   'channels_selected': array(['P3', 'F4', 'P4', 'Pz', 'O1', 'F7', 'F8', 'A2', 'T6'], dtype='<U3'),
│   'channels_idx_selected': [0, 4, 6, 8, 13, 15, 16, 17, 18],
│   'model': <keras.src.engine.functional.Functional object at 0x7f730a624390>,
│   'test_acc': 0.6818181872367859
}

In [9]:
X, y, _ = data_generator(fat_dataset, subjects=[model_info["subject"]], channel_idx=model_info["channels_idx_selected"], sfreq=model_info["sfreq"])
y_encoded = LabelEncoder().fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=SKLRNG, shuffle=True, stratify=y_encoded)

model_info["model"].compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
baseline_model_test = model_info["model"].evaluate(X_test, y_test, batch_size=model_info["batch_size"])

baseline_test_acc = baseline_model_test[1]

Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns


2024-03-24 15:26:17.354949: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-03-24 15:26:18.666268: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-24 15:26:18.972761: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 4s 4s/step - loss: 10.9936 - accuracy: 0.6818


In [10]:
_, keras_file = None, "./temp_model_export/baseline_model.h5"
tf.keras.models.save_model(model_info["model"], keras_file, include_optimizer=False)
rprint('Saved baseline model to:', keras_file)

/tmp/ipykernel_188098/3390911264.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_info["model"], keras_file, include_optimizer=False)


Saved baseline model to: ./temp_model_export/baseline_model.h5

In [11]:
# # temp_model = tf.keras.Model(
# #     inputs=model_info["model"].inputs,
# #     outputs=model_info["model"].outputs
# # )
# # temp_model.set_weights(model.user_attrs["trial_data"]["weights"])

# isinstance(model_info["model"], tf.keras.Model), model_info["model"]._is_graph_network

In [12]:
# np.ceil(60000 / 128).astype(np.int32) * 2

# np.ceil(len(X_train) / model_info["batch_size"]).astype(np.int32) * epochs

# 60000 / 128

# len(X_train) / model_info["batch_size"]

X_train.shape[0]

86

In [12]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = model_info["batch_size"]
epochs = 5
end_step = np.ceil(len(X_train) / batch_size).astype(np.int32) * epochs
# end_step = model_info["batch_size"]
# Define model for pruning.
pruning_params = {
    #   'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.9,
    #                                                           begin_step=0,
    #                                                           frequency=100)
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step)
}
keras.utils.get_custom_objects().update({
    **CUSTOM_OBJECTS
})
baseline_model_copy = tf.keras.models.clone_model(model_info["model"])
model_for_pruning = prune_low_magnitude(baseline_model_copy, **pruning_params)
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_for_pruning.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 9, 601, 1)]       0         
                                                                 
 prune_low_magnitude_conv2d  (None, 9, 579, 70)        3292      
 _20 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 1, 579, 30)        37802     
 _21 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_batch_  (None, 1, 579, 30)        121       
 normalization_10 (PruneLow                                      
 Magnitude)                                                      
                                                                 
 prune_low_magnitude_activa  (None, 1, 579, 30)        1  

In [13]:
logdir = "logs/pruning/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(X_train, y_train,
                  batch_size=model_info["batch_size"], 
                  epochs=epochs, 
                  validation_split=0.2,
                  callbacks=callbacks)

/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.GlorotUniform'>, which may lead to improper serialization.
  warnings.warn(
/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.Zeros'>, which may lead to improper serialization.
  warnings.warn(
/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/keras/src/regularizers.py:426: UserWarning: The `keras.regularizers.serialize()` API should only be used for o

Epoch 1/5


2024-03-24 15:28:00.850725: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/prune_low_magnitude_dropout_10/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-24 15:28:04.678412: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7143cbd890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-24 15:28:04.678492: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2024-03-24 15:28:04.685524: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711315684.794497  188258 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 12s 12s/step - loss: 14.7465 - accuracy: 0.4559 - val_loss: 13.7009 - val_accuracy: 0.2778
Epoch 2/5
1/1 [==============================] - 0s 121ms/step - loss: 13.4451 - accuracy: 0.5882 - val_loss: 13.3463 - val_accuracy: 0.2778
Epoch 3/5
1/1 [==============================] - 0s 113ms/step - loss: 13.1472 - accuracy: 0.6029 - val_loss: 12.9814 - val_accuracy: 0.3333
Epoch 4/5
1/1 [==============================] - 0s 118ms/step - loss: 12.7086 - accuracy: 0.6765 - val_loss: 12.6154 - val_accuracy: 0.3889
Epoch 5/5
1/1 [==============================] - 0s 120ms/step - loss: 12.2913 - accuracy: 0.7500 - val_loss: 12.2496 - val_accuracy: 0.3889


In [14]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(X_test, y_test, verbose=0)

rprint('Baseline test accuracy:', baseline_test_acc)
rprint('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.6818181872367859

Pruned test accuracy: 0.5909090638160706

In [15]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = None, "./temp_model_export/pruned_model.h5"
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
rprint('Saved pruned Keras model to:', pruned_keras_file)

/tmp/ipykernel_188098/3865207160.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: ./temp_model_export/pruned_model.h5

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = None, "./temp_model_export/pruned_model.tflite"
# _, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

rprint('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpxv1ejywf/assets


INFO:tensorflow:Assets written to: /tmp/tmpxv1ejywf/assets
2024-03-24 15:28:15.125403: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-24 15:28:15.125452: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-24 15:28:15.125775: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxv1ejywf
2024-03-24 15:28:15.127030: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-24 15:28:15.127053: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpxv1ejywf
2024-03-24 15:28:15.131000: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-03-24 15:28:15.131972: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-24 15:28:15.159885: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

Saved pruned TFLite model to: ./temp_model_export/pruned_model.tflite

In [17]:
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    import os
    import zipfile
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    
    return os.path.getsize(zipped_file)

rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
rprint("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
rprint("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 86034.00 bytes

Size of gzipped pruned Keras model: 85907.00 bytes

Size of gzipped pruned TFlite model: 83934.00 bytes

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = None, "./temp_model_export/quantized_and_pruned_model.tflite"

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

rprint('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

rprint("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
rprint("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmphwzmpi90/assets


INFO:tensorflow:Assets written to: /tmp/tmphwzmpi90/assets
2024-03-24 15:29:05.264027: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-03-24 15:29:05.264091: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-03-24 15:29:05.264509: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmphwzmpi90
2024-03-24 15:29:05.266426: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-03-24 15:29:05.266507: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmphwzmpi90
2024-03-24 15:29:05.274695: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-03-24 15:29:05.298239: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmphwzmpi90
2024-03-24 15:29:05.315031: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

Saved quantized and pruned TFLite model to: ./temp_model_export/quantized_and_pruned_model.tflite

Size of gzipped baseline Keras model: 86034.00 bytes

Size of gzipped pruned and quantized TFlite model: 24794.00 bytes

In [47]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # signatures = interpreter.get_signature_list()
  # rprint(interpreter.get_input_details(), interpreter.get_output_details(), signatures)

  # Run predictions on every image in the "test" dataset.
  predictions = []
  for i, v in enumerate(X_test):
    v = v[np.newaxis, :, :, np.newaxis].astype(np.float32)
    # if i % 1000 == 0:
    #   rprint('Evaluated on {n} results so far.'.format(n=i))
    # # Pre-processing: add batch dimension and convert to float32 to match with
    # # the model's input data format.
    # v = np.expand_dims(v, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, v)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    class_prediction = np.argmax(output()[0]) # 0 = left, 1 = right
    predictions.append(class_prediction)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  predictions = np.asarray(predictions)
  accuracy = (predictions == y_test).mean()
  return accuracy

interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

rprint('Pruned and quantized TFLite test_accuracy:', test_accuracy)
rprint('Pruned TF test accuracy:', model_for_pruning_accuracy)

Pruned and quantized TFLite test_accuracy: 0.5909090909090909

Pruned TF test accuracy: 0.5909090638160706